* OpenAI Whisper 可以把影片加字幕，但研究、訪談採取逐字稿的情景則不太合用，因無法辨識說話者，不會加標點。
* 這個程式用 Whisper 把語音轉正體中文逐字稿、每句結束加上全形逗號（但只會加逗號），再儲存成「原檔名_output.txt」
* 你可把文字檔下載後，截去不需要的部分、分段、加上合適標點（句號、驚歎號等）。

# 安裝及檢查必要的程式庫

按下「shift + Enter」它會執行這一格並跳到下一格，完成執行後左方會顯示數字而不是箭頭，這些是使用到的程式庫，如果沒安裝，它檢查完會去安裝，如果已經安裝了很快就跳到下一格。

In [ ]:
# 安裝必要的程式庫
%pip install -q openai-whisper ffmpeg-python opencc_python_reimplemented pydub
# 更新程式庫
%pip install --upgrade -q openai-whisper ffmpeg-python opencc_python_reimplemented pydub

In [ ]:
# 檢查是否安裝成功
import importlib

packages = {
    "openai-whisper": "whisper",
    "ffmpeg-python": "ffmpeg",
    "opencc-python-reimplemented": "opencc",
    "pydub": "pydub"
}

for pkg, import_name in packages.items():
    try:
        importlib.import_module(import_name)
        print(f"{pkg} is installed as {import_name}.")
    except ImportError:
        print(f"{pkg} is not installed.")


# 上傳檔案及找到檔案路徑

1. 把你要轉換的原檔案存入「convert_files」檔案夾之下。
    * 如果你在本機使用 Anaconda/ JupyterLab/ VSCode，就直接把檔案放進「convert_files」檔案夾中；
    * 如果在 Colab 等雲端，從側邊欄上傳檔案到「convert_files」檔案夾中。 
2. 這個程式設定用 MP3 格式，如果不是 MP3，它會轉換 WAV（錄音筆）, MP4（影音檔）, M4A（iPhone/ Mac 錄音檔） 這幾種常用格式，其他沒測過。
3. 確定檔案擺好後執行這一格，你的檔案路徑會顯示在下面（例如：r"./convert_files/filename.mp3"），整行複製下來。

In [ ]:
import os
from pathlib import Path

# Root dir
root_directory = Path(".")  # 設定根目錄為當前工作目錄

# travel all the dirs/ files
for file in root_directory.glob("convert_files/**/*"):
    if file.is_file():
        # 使用 r"" 來表示原始字串
        relative_path = file.relative_to(root_directory)
        # 將 \ 轉換為 /
        relative_path_str = str(relative_path).replace("\\", "/")
        print(f"r\"{relative_path_str}\"")

把你複製的待轉檔的檔案路徑貼到下面這一格，在「input_file_path =」的「=」（等號）後全部貼上。

例如「input_file_path = r"convert_files/file_name.mp4"」，等號後包括 r、前後引號

In [ ]:
input_file_path = r"convert_files/file_to_convert.wav"

# 轉檔為 MP3

偵測輸入檔案是影片還是聲音，分別用不同函式來轉換成 MP3，如果已經是 MP3 就不會動。

In [ ]:
import os
from pydub import AudioSegment
import subprocess
import ffmpeg  # 將 ffmpeg 直接導入這裡

def convert_file(input_file_path):
    # 取得檔案的副檔名
    _, ext = os.path.splitext(input_file_path)
    
    # 根據副檔名選擇轉換方式
    if ext.lower() == '.mp3':
        print(f"{input_file_path} 是 MP3 檔案，不需要轉換。")
        return input_file_path
    elif ext.lower() == '.mp4':
        print(f"{input_file_path} 是 MP4 檔案，進行影片轉換。")
        output_file_path = convert_video_to_mp3(input_file_path)
        return output_file_path
    elif ext.lower() in ['.m4a', '.wav']:
        print(f"{input_file_path} 是 {ext.upper()} 檔案，進行音訊轉換。")
        output_file_path = convert_audio_to_mp3(input_file_path)
        return output_file_path
    else:
        print(f"不支持的檔案格式：{ext}，無法進行轉換。")
        return None

def convert_video_to_mp3(input_file_path):
    base, _ = os.path.splitext(input_file_path)
    output_file_path = f"{base}_converted.mp3"
    
    # 使用 subprocess 調用 ffmpeg 命令進行轉換
    command = f"ffmpeg -i {input_file_path} -vn -ar 44100 -ac 2 -b:a 192k {output_file_path}"
    subprocess.call(command, shell=True)
    
    return output_file_path

def convert_audio_to_mp3(input_file):
    base_name = os.path.splitext(input_file)[0]
    output_file = f"{base_name}_converted.mp3"

    # 如果已是 mp3，就不轉換
    if input_file.lower().endswith('.mp3'):
        return input_file

    # 轉換為 mp3
    audio = AudioSegment.from_file(input_file)
    audio.export(output_file, format="mp3")
    return output_file

執行下面這格進行轉檔，不論檔案格式，執行就對了。

In [ ]:
converted_file= convert_file(input_file_path)

# 進行聽打

執行下一格會開始聽打，如果錄音很長，會需要一陣子，請耐心等待。

In [ ]:
import whisper
import time
from IPython.display import display, clear_output
import os
import opencc

# 載入 Whisper 模型
model = whisper.load_model("medium")  # small / medium / large

# 記錄開始時間
start_time = time.time()

# 進行聽打，指定語言為中文（不然它會翻譯成英文）
try:
    result = model.transcribe(converted_file, language="zh")  # 使用转换后的文件路径
except Exception as e:
    clear_output(wait=True)
    display(f"識別過程中出現錯誤: {str(e)}")
    raise

# 記錄結束時間及總執行時間
end_time = time.time()
elapsed_time = end_time - start_time
display(f"識別過程共花費了 {elapsed_time:.2f} 秒")

# 獲取轉錄的文本和分段信息
recognized_text = result["text"]
segments = result.get("segments", [])

# 用 opencc 將簡中轉換為繁中（default 是簡中）
converter = opencc.OpenCC('s2t')
recognized_text = converter.convert(recognized_text)

# 在每個段落結尾添加中文全形逗號
new_recognized_text = ""
for segment in segments:
    # 獲取段落文本
    segment_text = segment["text"]
    # 將段落文本轉換為繁體中文
    segment_text = converter.convert(segment_text)
    # 在段落尾部添加全形逗號
    new_recognized_text += segment_text + "，"

# 打印添加全形逗號後的文本
print("添加全形逗號後的文本：")
print(new_recognized_text)

# 生成輸出文件路徑
input_file_name = os.path.basename(input_file_path)  # 獲取輸入文件名
input_file_name_without_ext = os.path.splitext(input_file_name)[0]  # 去除擴展名
output_file_name = f"{input_file_name_without_ext}_output.txt"  # 生成輸出文件名
output_file_path = os.path.join(os.path.dirname(input_file_path), output_file_name)  # 生成完整的輸出文件路徑

# 將文本保存到 txt 文件
with open(output_file_path, "w", encoding="utf-8") as file:
    file.write(new_recognized_text)

print("文本已保存到", output_file_path)

# 下載完成的文字檔
最後，從左側邊欄的「convert_files」檔案夾中下載「原檔名_output.txt」。